In [3]:
import os
import sys
from dotenv import load_dotenv
import textwrap
from IPython.display import display
from IPython.display import Markdown

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY= os.getenv("AZURE_OPENAI_API_KEY")
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [8]:
from pathlib import Path
import openai
openai.api_type="openai"
api_key=OPENAI_API_KEY
speech_file_path = "speech.mp3"
response = openai.audio.speech.create(

  model="tts-1",
  voice="nova",
  input="안녕하세요? 원하시는 서비스를 말씀해주세요!"
)
response.stream_to_file(speech_file_path)


/tmp/ipykernel_21916/1579316434.py:12: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


#### Streaming real time audio

https://platform.openai.com/docs/guides/text-to-speech/streaming-real-time-audio

## voice class

### STT -> TTS 순으로 개발

Text To Text

Speech To Text

### TTS

In [1]:
from openai import OpenAI

client = OpenAI()
#api 호출
#audio 저장
text_input="""학사 안내
학적에 대한 답변
자퇴는 학생이 자발적으로 학교를 그만두는 것을 말합니다. 자퇴를 신청하려면 학사관리팀에 자퇴 신청서를 제출해야 하며,
자퇴가 승인되면 학적이 말소됩니다. 자퇴 후 재입학을 원할 경우, 재입학 절차를 따르게 됩니다.
자퇴에 대해서 주의할 점은 자퇴 후에는 학적이 말소되어 학업을 이어나갈 수 없으며, 재입학을 원할 경우에는 별도의 절차를 거쳐야 한다는 것입니다.

수강에 대한 답변
계절수업은 정규 학기 외에 여름방학과 겨울방학 동안 진행되는 수업을 말합니다. 계절수업은 학점 취득을 위해 개설되며, 학생들은 이를 통해 학점을 추가로 취득할 수 있습니다.

계절수업은 정규 학기와 동일한 방식으로 진행되며, 수강신청, 강의평가, 시험 및 성적 평가 등이 포함됩니다. 계절수업을 통해 학점을 취득하면 졸업 요건을 충족하는 데 도움이 됩니다.

계절수업에 대해서 주의할 점은 수강신청 기간과 수업 일정이 정규 학기와 다르므로, 이를 미리 확인하고 준비해야 한다는 것입니다. 또한, 계절수업은 추가 비용이 발생할 수 있습니다."""

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=text_input,
)

response.stream_to_file("output.mp3")

/tmp/ipykernel_36697/572254350.py:24: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("output.mp3")


### STT

In [2]:
client = OpenAI()
audio_path="/home/user/khtml-ai-api/audio_data/output.mp3"
transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=open(audio_path, "rb"),  
)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
    {"role": "system", "content":"""You are generating a transcript summary. Create a summary of the provided transcription. Respond in Markdown."""},
    {"role": "user", "content": [
        {"type": "text", "text": f"The audio transcription is: {transcription.text}"}
        ],  
    }
    ],
    temperature=0,
)
res=response.choices[0].message.content
print(res)

### Transcript Summary

#### 학적에 대한 답변 (Academic Status Information)
- **자퇴 (Withdrawal)**: 
  - 자퇴는 학생이 자발적으로 학교를 그만두는 것을 의미합니다.
  - 자퇴를 신청하려면 학사 관리팀에 자퇴 신청서를 제출해야 하며, 승인되면 학적이 말소됩니다.
  - 자퇴 후 재입학을 원할 경우, 별도의 재입학 절차를 따라야 합니다.
  - 주의사항: 자퇴 후 학적이 말소되어 학업을 이어나갈 수 없으며, 재입학을 위해서는 별도의 절차가 필요합니다.

#### 수강에 대한 답변 (Course Enrollment Information)
- **계절 수업 (Seasonal Classes)**:
  - 계절 수업은 여름방학과 겨울방학 동안 진행되는 수업으로, 학점 취득을 위해 개설됩니다.
  - 정규학기와 동일한 방식으로 진행되며, 수강 신청, 강의평가, 시험 및 성적평가 등이 포함됩니다.
  - 계절 수업을 통해 추가 학점을 취득하면 졸업 요건을 충족하는 데 도움이 됩니다.
  - 주의사항: 수강 신청 기간과 수업 일정이 정규학기와 다르므로 미리 확인하고 준비해야 하며, 추가 비용이 발생할 수 있습니다.
